### From Instance Segmented Annotation to Bounding Box
**Dataset in focus:** Luderick Dataset


In [ ]:
import os
import glob

def convert_yolo_segmentation_to_bbox(seg_file_path, output_dir):
    """
    Reads a YOLO segmentation file, converts its polygons to bounding boxes,
    and saves the result to a new file.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the output file path
    base_filename = os.path.basename(seg_file_path)
    output_file_path = os.path.join(output_dir, base_filename)

    with open(seg_file_path, 'r') as f_in, open(output_file_path, 'w') as f_out:
        for line in f_in:
            parts = line.strip().split()
            if len(parts) < 3: # Must have at least a class and one point
                continue

            class_id = parts[0]
            # Get all coordinates, skipping the class_id
            coords = [float(p) for p in parts[1:]]

            # Separate into x and y coordinates
            x_coords = coords[0::2] # Every other element starting from index 0
            y_coords = coords[1::2] # Every other element starting from index 1

            # Find the min and max to get the bounding box
            x_min = min(x_coords)
            y_min = min(y_coords)
            x_max = max(x_coords)
            y_max = max(y_coords)

            # Calculate YOLO bounding box format (x_center, y_center, width, height)
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min
            x_center = x_min + (bbox_width / 2)
            y_center = y_min + (bbox_height / 2)

            # Write the new bounding box line to the output file
            f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}\n")

def process_directory(input_dir, output_dir):
    """
    Processes all .txt files in a directory.
    """
    # Find all .txt files in the input directory
    txt_files = glob.glob(os.path.join(input_dir, '*.txt'))
    
    if not txt_files:
        print(f"No .txt files found in {input_dir}")
        return
        
    print(f"Found {len(txt_files)} files to convert in {input_dir}.")

    for txt_file in txt_files:
        convert_yolo_segmentation_to_bbox(txt_file, output_dir)
        
    print(f"Conversion complete. Bounding box files saved in {output_dir}")

# 'train' set
input_labels_dir = '/Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/InstanceSegmented_Luderick_dataset_yolov11/train/labels'
output_labels_dir = '/Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/InstanceSegmented_Luderick_dataset_yolov11/train/labels_bbox' # Create a new folder for the converted files

process_directory(input_labels_dir, output_labels_dir)


Found 4276 files to convert in /Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/InstanceSegmented_Luderick_dataset_yolov11/train/labels.
Conversion complete. Bounding box files saved in /Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/InstanceSegmented_Luderick_dataset_yolov11/train/labels_bbox


### From XML to Yolo
**Dataset in focus:**
- F4K Detection and Tracking
- FishCLEF-2015

In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import cv2
from pathlib import Path
from tqdm import tqdm  # Import the tqdm library

# --- Configuration ---
F4K_DATASET_PATH = "/Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/f4k_detection_tracking"
FISHCLEF_DATASET_PATH = "/Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/fishclef_2015_release"
# --- End of Configuration ---

def convert_bbox_to_yolo(box, img_w, img_h):
    """Converts a bounding box [x_min, y_min, x_max, y_max] to YOLO format."""
    x_min, y_min, x_max, y_max = box
    dw = 1. / img_w
    dh = 1. / img_h
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0
    w = x_max - x_min
    h = y_max - y_min
    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh
    return f"0 {x:.6f} {y:.6f} {w:.6f} {h:.6f}"

def process_f4k_dataset(base_path):
    """Processes the f4k_detection_tracking dataset."""
    print(f"\n--- Processing F4K Dataset in: {base_path} ---")
    xml_dir = Path(base_path)
    images_dir = Path(base_path) / "images"
    labels_dir = Path(base_path) / "labels"
    labels_dir.mkdir(exist_ok=True)
    
    xml_files = glob.glob(str(xml_dir / "*.xml"))
    if not xml_files:
        print(f"Error: No XML files found in {xml_dir}.")
        return

    video_dims_cache = {}

    # --- TQDM Progress Bar Added Here ---
    for xml_file in tqdm(xml_files, desc="Processing F4K XMLs"):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        video_name = os.path.basename(xml_file).replace('.xml', '')

        for frame in root.findall('frame'):
            frame_id = int(frame.get('id'))
            image_filename = f"{video_name}_frame_{frame_id:05d}.jpg"
            image_path = images_dir / image_filename
            
            if not image_path.exists():
                continue

            if video_name not in video_dims_cache:
                img = cv2.imread(str(image_path))
                if img is None: continue
                img_h, img_w, _ = img.shape
                video_dims_cache[video_name] = (img_w, img_h)
            img_w, img_h = video_dims_cache[video_name]

            yolo_annotations = []
            for obj in frame.findall('object'):
                if obj.get('objectType') == 'fish':
                    contour_text = obj.find('contour').text
                    points = [tuple(map(int, p.split(' '))) for p in contour_text.strip().split(',')]
                    if not points: continue
                    
                    x_coords = [p[0] for p in points]
                    y_coords = [p[1] for p in points]
                    x_min, x_max = min(x_coords), max(x_coords)
                    y_min, y_max = min(y_coords), max(y_coords)
                    
                    yolo_annotations.append(convert_bbox_to_yolo([x_min, y_min, x_max, y_max], img_w, img_h))

            if yolo_annotations:
                label_filename = f"{video_name}_frame_{frame_id:05d}.txt"
                with open(labels_dir / label_filename, 'w') as f:
                    f.write("\n".join(yolo_annotations))
    print("--- F4K Processing Complete ---")

def process_fishclef_dataset(base_path):
    """Processes the fishclef_2015_release dataset."""
    print(f"\n--- Processing FishCLEF Dataset in: {base_path} ---")
    
    images_dir = Path(base_path) / "all_frames"
    labels_dir = Path(base_path) / "labels"
    labels_dir.mkdir(exist_ok=True)
    
    xml_files = glob.glob(str(Path(base_path) / "**" / "gt" / "*.xml"), recursive=True)
    if not xml_files:
        print(f"Error: No XML files found in {base_path}.")
        return

    video_dims_cache = {}

    # --- TQDM Progress Bar Added Here ---
    for xml_file in tqdm(xml_files, desc="Processing FishCLEF XMLs"):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        video_base_name = Path(xml_file).stem

        for frame in root.findall('frame'):
            frame_id = int(frame.get('id'))
            image_filename = f"{video_base_name}_frame_{frame_id:05d}.jpg"
            image_path = images_dir / image_filename

            if not image_path.exists():
                # Some videos in FishCLEF have slightly different frame naming conventions,
                # so we search for a matching pattern if the exact name isn't found.
                potential_files = list(images_dir.glob(f"{video_base_name}_frame_{frame_id:05d}*.jpg"))
                if not potential_files:
                    continue
                image_path = potential_files[0]
            
            if video_base_name not in video_dims_cache:
                img = cv2.imread(str(image_path))
                if img is None: continue
                img_h, img_w, _ = img.shape
                video_dims_cache[video_base_name] = (img_w, img_h)
            img_w, img_h = video_dims_cache[video_base_name]
            
            yolo_annotations = []
            for obj in frame.findall('object'):
                x_min = int(obj.get('x'))
                y_min = int(obj.get('y'))
                w = int(obj.get('w'))
                h = int(obj.get('h'))
                x_max = x_min + w
                y_max = y_min + h
                
                yolo_annotations.append(convert_bbox_to_yolo([x_min, y_min, x_max, y_max], img_w, img_h))
            
            if yolo_annotations:
                label_filename = f"{image_path.stem}.txt"
                with open(labels_dir / label_filename, 'w') as f:
                    f.write("\n".join(yolo_annotations))
    print("--- FishCLEF Processing Complete ---")


if __name__ == "__main__":
    if not Path(F4K_DATASET_PATH).exists() or not Path(FISHCLEF_DATASET_PATH).exists():
        print("Error: One or both dataset paths in the configuration are incorrect. Please update them.")
    else:
        process_f4k_dataset(F4K_DATASET_PATH)
        process_fishclef_dataset(FISHCLEF_DATASET_PATH)
        print("\nAll conversions finished!")


--- Processing F4K Dataset in: /Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/f4k_detection_tracking ---


Processing F4K XMLs: 100%|██████████| 17/17 [00:01<00:00, 13.86it/s]


--- F4K Processing Complete ---

--- Processing FishCLEF Dataset in: /Users/marco/Desktop/BlueOASIS/SmartFISHER/sff_dataset/fishclef_2015_release ---


Processing FishCLEF XMLs: 100%|██████████| 93/93 [00:08<00:00, 11.36it/s]

--- FishCLEF Processing Complete ---

All conversions finished!
